In [1]:
# Importing modules
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to /Users/aruvi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('movies_filled_cleaned_processed.csv')

data.head()

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,release_date_filled,release_year,running_time_mins
0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,"December 16, 2015",2015,138
1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,"April 24, 2019",2019,181
2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,"December 16, 2009",2009,162
3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,"January 29, 2018",2018,134
4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,"April 23, 2018",2018,149


In [3]:
text_data = data[["Movie Info"]]

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Remove punctuation
text_data['Movie_Info_clean'] = \
text_data['Movie Info'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
text_data['Movie_Info_clean'] = \
text_data['Movie_Info_clean'].map(lambda x: x.lower())
# Print out the first rows of papers
text_data['Movie_Info_clean'].head()

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-4-5c9f4ba9be45>:3: DeprecationWarning: invalid escape sequence \.
  text_data['Movie Info'].map(lambda x: re.sub('[,\.!?]', '', x))
<ipython-input-4-5c9f4ba9be45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['Movie_Info_clean'] = \
<ipython-input-4-5c

0    as a new threat to the galaxy rises rey a dese...
1    after the devastating events of avengers: infi...
2    a paraplegic marine dispatched to the moon pan...
3    t'challa heir to the hidden but advanced kingd...
4    the avengers and their allies must be willing ...
Name: Movie_Info_clean, dtype: object

In [5]:
#pip install pyLDAvis

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
stop_words = stopwords.words('english')
stop_words.extend(['us','get','one','two','stop','from', 'subject', 're', 'for','then','must'])
def sent_to_words(para):
    for p in para:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(p), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
info = text_data.Movie_Info_clean.values.tolist()
words = list(sent_to_words(info))
# remove stop words
words = remove_stopwords(words)
words[0][1:10]

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['threat',
 'galaxy',
 'rises',
 'rey',
 'desert',
 'scavenger',
 'finn',
 'ex',
 'stormtrooper']

In [13]:
id2word = corpora.Dictionary(words)

texts = words

corpus = [id2word.doc2bow(text) for text in texts]


/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:


num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.010*"new" + 0.006*"team" + 0.006*"world" + 0.005*"find" + 0.005*"two" + '
  '0.004*"city" + 0.004*"home" + 0.003*"york" + 0.003*"sent" + 0.003*"earth"'),
 (1,
  '0.008*"life" + 0.007*"new" + 0.007*"family" + 0.006*"world" + 0.005*"war" + '
  '0.004*"friends" + 0.004*"young" + 0.003*"old" + 0.003*"school" + '
  '0.003*"love"'),
 (2,
  '0.006*"man" + 0.006*"young" + 0.006*"new" + 0.005*"woman" + 0.005*"two" + '
  '0.005*"city" + 0.005*"world" + 0.004*"time" + 0.004*"family" + '
  '0.004*"life"'),
 (3,
  '0.005*"team" + 0.004*"world" + 0.004*"earth" + 0.004*"time" + 0.003*"woman" '
  '+ 0.003*"american" + 0.003*"becomes" + 0.003*"trying" + 0.003*"battle" + '
  '0.003*"murder"'),
 (4,
  '0.009*"young" + 0.008*"man" + 0.007*"world" + 0.005*"team" + 0.004*"new" + '
  '0.004*"two" + 0.003*"order" + 0.003*"life" + 0.003*"rescue" + '
  '0.003*"earth"'),
 (5,
  '0.006*"new" + 0.004*"parents" + 0.004*"help" + 0.004*"dangerous" + '
  '0.004*"young" + 0.004*"years" + 0.003*"save" + 0.003*

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.086405 -0.027896       1        1  11.858675
1     -0.026193 -0.091275       2        1  11.380869
0      0.069102  0.033640       3        1  11.128389
4     -0.040085  0.051549       4        1  11.061995
7     -0.028620  0.018721       5        1  10.805814
9      0.089857  0.011300       6        1   9.985426
8      0.069338 -0.099718       7        1   9.407229
3      0.027631  0.064303       8        1   8.930426
5     -0.020485  0.051413       9        1   7.816365
6     -0.054140 -0.012038      10        1   7.624812, topic_info=          Term       Freq      Total Category  logprob  loglift
8          new  80.000000  80.000000  Default  30.0000  30.0000
181      young  49.000000  49.000000  Default  29.0000  29.0000
205       time  28.000000  28.000000  Default  28.0000  28.0000
140       team  38.000000  38.000000  Default  27.0000  27.0000
314       city  34.000000  34.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
1282  american   2.351730  20.292165  Topic10  -6.1426   0.4187
518       take   2.309195  20.262691  Topic10  -6.1608   0.4019
49       world   2.647154  72.632855  Topic10  -6.0243  -0.7382
25        help   2.418473  36.379232  Topic10  -6.1146  -0.1371
27       order   2.248326  29.237674  Topic10  -6.1876   0.0085

[808 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1614      9  0.704931   abraham
2414      5  0.677968  absconds
4809      5  0.677964    absent
1168      3  0.307658  accident
1168      6  0.307658  accident
...     ...       ...       ...
181       9  0.081507     young
181      10  0.081507     young
2425      5  0.899893    yuppie
2614      1  0.272027      zeus
2614      4  0.544054      zeus

[1835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 5, 8, 10, 9, 4, 6, 7])

In [16]:
pyLDAvis.save_html(vis, 'lda.html')

/Users/aruvi/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
